In [1]:
# default_exp viseme_tabular.data

# Viseme tabular data

> Tabular data handling functions.

TODO: create some test/data/capture_session ... so we can test end-to-end quickly

In [51]:
#export
from expoco.core import *
from expoco.camera_capture import *
from pathlib import Path
import numpy as np
import pandas as pd
import json, cv2

import mediapipe as mp
mp_face_mesh = mp.solutions.face_mesh

In [5]:
#export
def viseme_tabular_dataset_from_capture_sessions(
        input_path='data/capture_sessions', 
        glob_pattern='*'):
    "Create a viseme tabular dataset from capture session images"
    input_path, dataset_id, data = Path(input_path), now(), []
    output_path = input_path.parent/f'viseme_tabular_dataset_{dataset_id}'
    output_path.mkdir()
    face_mesh = mp_face_mesh.FaceMesh(max_num_faces=1)
    landmark_ids = sorted(FaceLandmarks.pointer + FaceLandmarks.mouth)
    metadata = dict(input_path=path_to_str(input_path), output_path=path_to_str(output_path), 
                    glob_pattern=glob_pattern, session_metadata=[], start_date=now(), 
                    landmark_ids=landmark_ids,
                    column_names=landmark_ids_to_col_names(FaceLandmarks.pointer + FaceLandmarks.mouth))
    data = []
    for session_path in sorted(input_path.glob(glob_pattern)):
        with open(session_path/'metadata.json') as f:
            session_metadata = json.load(f)
        metadata['session_metadata'].append(session_metadata)
        for capture_count in range(1, session_metadata['count']+1):
            image = cv2.imread(f'{session_path}/{capture_count}.png')
            results = face_mesh.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
            row = []
            for landmark_id in landmark_ids:
                landmark = results.multi_face_landmarks[0].landmark[landmark_id]
                for coord in ['x','y']:
                    row.append(getattr(landmark, coord))
            data.append(row)
    metadata['end_date'] = now()
    with open(output_path/'metadata.json', 'w') as f: json.dump(metadata, f, indent=2)
    np.save(output_path/'data.npy', np.array(data), allow_pickle=False)
    return output_path

In [6]:
_viseme_tabular_dataset_path = viseme_tabular_dataset_from_capture_sessions()
_viseme_tabular_dataset_path

WindowsPath('data/viseme_tabular_dataset_20211130_163506')

In [8]:
#export
def read_viseme_tabular_dataset(dataset_path, to_numpy=True):
    "Return metadata and data (as a numpy array by default, pandas dataframe if `to_numpy=False`)"
    dataset_path = Path(dataset_path)
    data = np.load(dataset_path/'data.npy')
    with open(dataset_path/'metadata.json') as f:
        dataset_metadata = json.load(f)
    viseme_class = []
    for session_metadata in dataset_metadata['session_metadata']:
        viseme_class.extend([get_viseme_class(session_metadata)]*session_metadata['count'])
    if to_numpy: 
        return dataset_metadata, data, viseme_class
    df = pd.DataFrame(data, columns=dataset_metadata['column_names'])
    df = pd.concat([df, pd.DataFrame(dict(viseme_class=viseme_class))], axis=1)
    return dataset_metadata, df

In [9]:
_viseme_tabular_dataset = read_viseme_tabular_dataset(_viseme_tabular_dataset_path)
print(_viseme_tabular_dataset[0]['column_names'][:3],
      _viseme_tabular_dataset[1].shape, _viseme_tabular_dataset[1].dtype,
      len(_viseme_tabular_dataset[2]), 
      _viseme_tabular_dataset[2][:3])

['0x', '0y', '1x'] (5000, 214) float64 5000 ['NO_EXPRESSION', 'NO_EXPRESSION', 'NO_EXPRESSION']


In [10]:
_viseme_tabular_dataset = read_viseme_tabular_dataset(_viseme_tabular_dataset_path, False)
_viseme_tabular_dataset[1]

,0x,0y,1x,1y,2x,2y,5x,5y,11x,11y,...,409y,410x,410y,415x,415y,424x,424y,438x,438y,viseme_class
0,0.585625,0.660046,0.588794,0.600464,0.586255,0.618263,0.588732,0.559518,0.585293,0.668359,...,0.675027,0.636371,0.663649,0.615994,0.676276,0.629503,0.713615,0.612580,0.597352,NO_EXPRESSION
1,0.585041,0.662891,0.587921,0.602483,0.585833,0.620273,0.588698,0.560626,0.584704,0.671358,...,0.681065,0.634902,0.669131,0.614619,0.681139,0.628074,0.719766,0.611412,0.600389,NO_EXPRESSION
2,0.584732,0.664500,0.586855,0.603078,0.585257,0.620593,0.587649,0.560905,0.584496,0.673194,...,0.681760,0.635803,0.669330,0.615534,0.682209,0.629402,0.720872,0.611002,0.600284,NO_EXPRESSION
3,0.584848,0.663892,0.587416,0.601015,0.585749,0.619330,0.588384,0.559342,0.584573,0.672633,...,0.682110,0.636785,0.669808,0.616257,0.682514,0.630233,0.721648,0.611792,0.598807,NO_EXPRESSION
4,0.585212,0.664043,0.587976,0.599984,0.586221,0.618667,0.588973,0.558423,0.584901,0.672765,...,0.681375,0.637322,0.669009,0.617027,0.681913,0.630324,0.720235,0.612511,0.597646,NO_EXPRESSION
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,0.494891,0.714446,0.492595,0.656488,0.494302,0.670536,0.492934,0.608565,0.494982,0.723624,...,0.724569,0.556361,0.710953,0.533755,0.725643,0.552097,0.765974,0.521974,0.648878,RANDOM_TALK
4996,0.493530,0.714595,0.491592,0.654348,0.493174,0.669053,0.492149,0.606404,0.493566,0.724084,...,0.725426,0.555080,0.711338,0.532259,0.726595,0.550442,0.766505,0.521238,0.646845,RANDOM_TALK
4997,0.494015,0.713840,0.491472,0.652695,0.493215,0.667465,0.492036,0.605348,0.494109,0.723420,...,0.725037,0.555575,0.710510,0.532905,0.726259,0.551191,0.765363,0.521117,0.645328,RANDOM_TALK
4998,0.490585,0.710929,0.487390,0.653454,0.489662,0.667242,0.487971,0.605584,0.490739,0.720230,...,0.722074,0.552369,0.707964,0.529524,0.723374,0.548667,0.763049,0.517271,0.645648,RANDOM_TALK


In [11]:
#export
def get_image_path(dataset_path, row_idx):
    dataset_path = Path(dataset_path)
    if dataset_path.name.startswith('processed'):
        # TODO: read processed metadata and find dataset path - don't just assume its the parent
        dataset_path = dataset_path.parent
    with open(dataset_path/'metadata.json') as f:
        dataset_metadata = json.load(f)
    idx = row_idx
    for session_metadata in dataset_metadata['session_metadata']:
        if idx < session_metadata['count']:
            return Path(session_metadata['path'])/f'{idx+1}.jpeg'
        idx -= session_metadata['count']

In [12]:
get_image_path(_viseme_tabular_dataset_path, 200)

WindowsPath('data/capture_sessions/20211130_120029/1.jpeg')

In [13]:
#exporti
def select_columns(dataset_metadata, data, column_names):
    return data[..., [c in column_names for c in dataset_metadata['column_names']]]

In [14]:
_dataset_metadata, _data, _viseme_class = read_viseme_tabular_dataset(_viseme_tabular_dataset_path, True)
assert (5000, 214) == _data.shape
_cont_names = landmark_ids_to_col_names(FaceLandmarks.pointer + FaceLandmarks.mouth, [FaceLandmarks.tip_of_nose])
assert len(_cont_names) == 212
_data = select_columns(_dataset_metadata, _data, _cont_names + ['expression_id'])
assert (5000, 212) == _data.shape

In [15]:
#export
def make_landmarks_relative(data, to_landmarks, column_count=None):
    "Make all landmarks in `data` relative `to_landmarks` in-place"
    coord_count = to_landmarks.shape[-1]
    assert coord_count == 2, f'to_landmarks must have 2 "coord_count". to_arr.shape={coord_count}'
    for i in range(coord_count):
        data[:, i:column_count:coord_count] -= to_landmarks[..., i][..., None]
    return data

Columns in `arr` must be ordered as pairs of landmark IDs. e.g. `'5x', '5y', '2x', '2y', '218x', '218y' ...`

- `arr` 2d np array (rows, columns)
- `to_arr` 2d np array (rows, exactly 2 columns; `x` then `y`)
    - `to_arr.shape[0]` must be the same as `arr.shape[0]` - i.e. they must have the same number of rows
    
TODO: we could allow `to_arr.shape[1]` to be 1, 2 or 3 but ... leave it as is for now - might be hard to debug data issues if we accidentaly pass 1 column when we should be passing 2.

In [16]:
_arr = np.array([[.1,.2,.3,.4,0],
                [.2,.2,.3,.4,1],
                [.2,.6,.3,.6,2]])
_to_arr = np.array([[.0,.0],
                   [.2,.4],
                   [.8,.9]])
make_landmarks_relative(_arr, _to_arr, _arr.shape[-1]-1)

array([[ 0.1,  0.2,  0.3,  0.4,  0. ],
       [ 0. , -0.2,  0.1,  0. ,  1. ],
       [-0.6, -0.3, -0.5, -0.3,  2. ]])

In [17]:
_arr = np.array([[.1,.2,.3,.4,0]])
_to_arr = np.array([[-.3,.8]])
make_landmarks_relative(_arr, _to_arr)

array([[ 0.4, -0.6,  0.6, -0.4,  0.3]])

In [18]:
_dataset_metadata, _data, _viseme_class = read_viseme_tabular_dataset(_viseme_tabular_dataset_path, True)
_cont_names = landmark_ids_to_col_names(FaceLandmarks.pointer + FaceLandmarks.mouth, [FaceLandmarks.tip_of_nose])
_cont_data = select_columns(_dataset_metadata, _data, _cont_names + ['expression_id'])
_tip_of_nose_columns = landmark_ids_to_col_names([FaceLandmarks.tip_of_nose])
_tip_of_nose_data = select_columns(_dataset_metadata, _data, _tip_of_nose_columns)
_relative_data = make_landmarks_relative(_cont_data, _tip_of_nose_data, len(_cont_names))
assert (5000, 212) == _relative_data.shape

In [19]:
pd.concat([pd.DataFrame(_relative_data, columns=_cont_names), pd.DataFrame(dict(viseme_class=_viseme_class))], axis=1)

,0x,0y,2x,2y,5x,5y,11x,11y,12x,12y,...,409y,410x,410y,415x,415y,424x,424y,438x,438y,viseme_class
0,-0.003169,0.059583,-0.002539,0.017800,-0.000062,-0.040946,-0.003501,0.067895,-0.004080,0.073906,...,0.074563,0.047578,0.063186,0.027200,0.075813,0.040709,0.113151,0.023786,-0.003111,NO_EXPRESSION
1,-0.002880,0.060409,-0.002088,0.017790,0.000777,-0.041857,-0.003217,0.068875,-0.003714,0.074870,...,0.078583,0.046981,0.066648,0.026698,0.078657,0.040153,0.117283,0.023491,-0.002094,NO_EXPRESSION
2,-0.002123,0.061423,-0.001598,0.017516,0.000793,-0.042172,-0.002359,0.070116,-0.002717,0.076309,...,0.078682,0.048947,0.066253,0.028679,0.079132,0.042547,0.117795,0.024146,-0.002794,NO_EXPRESSION
3,-0.002568,0.062877,-0.001667,0.018315,0.000968,-0.041673,-0.002843,0.071618,-0.003221,0.077896,...,0.081095,0.049370,0.068793,0.028841,0.081499,0.042817,0.120633,0.024376,-0.002208,NO_EXPRESSION
4,-0.002763,0.064058,-0.001755,0.018683,0.000997,-0.041561,-0.003074,0.072780,-0.003475,0.078958,...,0.081391,0.049347,0.069025,0.029051,0.081928,0.042348,0.120250,0.024536,-0.002339,NO_EXPRESSION
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,0.002296,0.057958,0.001707,0.014049,0.000338,-0.047922,0.002386,0.067137,0.002707,0.073054,...,0.068081,0.063765,0.054465,0.041160,0.069155,0.059501,0.109487,0.029379,-0.007610,RANDOM_TALK
4996,0.001938,0.060247,0.001581,0.014705,0.000557,-0.047945,0.001974,0.069736,0.002222,0.075986,...,0.071078,0.063488,0.056990,0.040666,0.072247,0.058849,0.112157,0.029646,-0.007503,RANDOM_TALK
4997,0.002542,0.061145,0.001743,0.014770,0.000564,-0.047346,0.002637,0.070725,0.002970,0.077140,...,0.072343,0.064103,0.057816,0.041433,0.073565,0.059719,0.112669,0.029645,-0.007367,RANDOM_TALK
4998,0.003195,0.057475,0.002272,0.013788,0.000581,-0.047870,0.003349,0.066776,0.003785,0.073096,...,0.068620,0.064979,0.054511,0.042134,0.069920,0.061277,0.109595,0.029880,-0.007806,RANDOM_TALK


In [20]:
#export
def change_viseme_class(viseme_class, from_class, to_class):
    "Change values in viseme_class from one class to another"
    return [to_class if c==from_class else c for c in viseme_class]

In [21]:
#export
def calculate_stats(data):
    "Return the mean and standard deviation over columns of `data`"
    return dict(mean=data.mean(axis=0), std=data.std(axis=0))

In [22]:
_dataset_metadata, _data, _viseme_class = read_viseme_tabular_dataset(_viseme_tabular_dataset_path, True)
_data = select_columns(_dataset_metadata, _data, _cont_names[:2])
_stats = calculate_stats(_data)
assert (2,) == _stats['mean'].shape
assert (2,) == _stats['std'].shape

In [23]:
#export
def normalize(data, mean, std):
    column_count = len(mean) # use len rather than shape[0] so that mean stats can be lists or np arrays
    data[..., :column_count] -= mean
    data[..., :column_count] /= std
    return data

In [24]:
_dataset_metadata, _data, _viseme_class = read_viseme_tabular_dataset(_viseme_tabular_dataset_path, True)
_data = select_columns(_dataset_metadata, _data, _cont_names[:2])
_stats = calculate_stats(_data)
_normalized_data = normalize(_data, **_stats)
assert np.allclose(_normalized_data.mean(axis=0), 0)
assert np.allclose(_normalized_data.std(axis=0), 1)

# Combine data processing steps

In [25]:
#export
def inference_data_from_landmarks(landmarks, landmark_ids, relative_landmark_id=None, coords=['x', 'y'], stats=None):
    landmark_ids, coords, data, to_landmarks_data = sorted(landmark_ids), sorted(coords), [], []
    for landmark_id in landmark_ids:
        for coord in coords:
            if landmark_id == relative_landmark_id:
                to_landmarks_data.append(getattr(landmarks[landmark_id], coord))
            else:
                data.append(getattr(landmarks[landmark_id], coord))
    data, to_landmarks_data = np.array([data], dtype=float), np.array([to_landmarks_data], dtype=float)
    if relative_landmark_id is not None:
        make_landmarks_relative(data, to_landmarks_data)
    if stats is not None:
        normalize(data, **stats)
    return data

In [26]:
from collections import namedtuple
_Landmark = namedtuple('_Landmark', ['x','y','z'])
_landmarks=[]
for i in range(FaceLandmarks.count):
    _landmarks.append(_Landmark(i+.1, i+.2, i+.3))
assert [[-1., -1.,  1.,  1.]] == inference_data_from_landmarks(_landmarks, [0,2,1], 1).round().tolist()

In [27]:
_stats = dict(mean=[0.1,0.2,0.3,0.4], std=[0.1,0.2,0.3,0.4])
_inference_data = inference_data_from_landmarks(
        landmarks=_landmarks, landmark_ids=[2,0,1], relative_landmark_id=1, stats=_stats)
assert [[-11.0,-6.0,2.3,1.5]] == np.round(_inference_data, 1).tolist()

In [42]:
#export
def processed_dataset_from_viseme_tabular_dataset(
        input_path, relative_landmark_id=None, change_y_from=None, change_y_to=None):
    "Create a processed dataset (ready for ML) from a viseme tabular dataset"
    input_path = Path(input_path)
    input_metadata, input_data, input_y = read_viseme_tabular_dataset(input_path)
    output_path = Path(input_metadata['output_path'])/f'processed_{now()}'
    output_path.mkdir()
    column_names = landmark_ids_to_col_names(input_metadata['landmark_ids'], relative_landmark_id)
    metadata = dict(input_path=path_to_str(input_path), 
                    input_metadata=input_metadata, 
                    output_path=path_to_str(output_path),
                    relative_landmark_id=relative_landmark_id, 
                    change_y_from=change_y_from,
                    change_y_to=change_y_to,
                    column_names=column_names,
                    start_date=now())
    data = select_columns(input_metadata, input_data, column_names)
    if change_y_from is not None:
        assert change_y_to is not None
        input_y = change_viseme_class(input_y, change_y_from, change_y_to)
    metadata['viseme_class'] = input_y
    if relative_landmark_id is not None:
        relative_names = landmark_ids_to_col_names([relative_landmark_id])
        metadata['relative_names']=relative_names
        to_landmarks = select_columns(input_metadata, input_data, relative_names)
        make_landmarks_relative(data, to_landmarks)
    stats = calculate_stats(data)
    normalize(data, **stats)
    metadata['end_date'] = now()
    with open(output_path/'metadata.json', 'w') as f: json.dump(metadata, f, indent=2)
    np.save(output_path/'data.npy', data, allow_pickle=False)
    np.savez(output_path/'stats.npz', **stats)
    return output_path

In [43]:
_processed_dataset_path = processed_dataset_from_viseme_tabular_dataset(
        input_path=_viseme_tabular_dataset_path, 
        relative_landmark_id=FaceLandmarks.tip_of_nose,
        change_y_from='RANDOM_TALK', change_y_to='NO_EXPRESSION')

In [44]:
_processed_dataset_path

WindowsPath('data/viseme_tabular_dataset_20211130_163506/processed_20211130_171906')

In [45]:
def to_processed_dataset_from_viseme_dataset_cli(
        input_path, relative_landmark_id=None, change_y_from=None, change_y_to=None):
    print(f"""processed_dataset_from_viseme_tabular_dataset ^
        {path_to_str(input_path)} ^
        --relative_landmark_id {relative_landmark_id} ^
        --change_y_from {change_y_from} ^
        --change_y_to {change_y_to}
    """)
to_processed_dataset_from_viseme_dataset_cli(
        input_path=_processed_dataset_path, 
        relative_landmark_id=FaceLandmarks.tip_of_nose,
        change_y_from='RANDOM_TALK', change_y_to='NO_EXPRESSION')

processed_dataset_from_viseme_tabular_dataset ^
        data/viseme_tabular_dataset_20211130_163506/processed_20211130_171906 ^
        --relative_landmark_id 1 ^
        --change_y_from RANDOM_TALK ^
        --change_y_to NO_EXPRESSION
    


In [46]:
#export
def read_processed_dataset(dataset_path, to_numpy=True):
    "Return metadata, data and stats (as a numpy array by default, pandas dataframe if `to_numpy=False`)"
    dataset_path = Path(dataset_path)
    data, stats = np.load(dataset_path/'data.npy'), np.load(dataset_path/'stats.npz')
    with open(dataset_path/'metadata.json') as f:
        dataset_metadata = json.load(f)
    viseme_class = dataset_metadata['viseme_class']
    if to_numpy: 
        return dataset_metadata, data, viseme_class, stats
    column_names = dataset_metadata['column_names']
    stats = dict(mean=pd.DataFrame(stats['mean'][None, ...], columns=column_names),
                 std=pd.DataFrame(stats['std'][None, ...], columns=column_names))
    df = pd.DataFrame(data, columns=dataset_metadata['column_names'])
    df = pd.concat([df, pd.DataFrame(dict(viseme_class=viseme_class))], axis=1)
    return dataset_metadata, df, stats

In [47]:
_metadata, _data, _viseme_class, _stats = read_processed_dataset(_processed_dataset_path, True)

In [50]:
_metadata, _data, _stats = read_processed_dataset(_processed_dataset_path, False)
_data

,0x,0y,2x,2y,5x,5y,11x,11y,12x,12y,...,409y,410x,410y,415x,415y,424x,424y,438x,438y,viseme_class
0,-0.368992,-0.210992,-0.422914,-0.248874,-0.935113,0.236309,-0.370399,-0.272868,-0.382151,-0.387187,...,-0.514617,-0.440148,-0.446618,-0.379710,-0.537136,-0.453986,-0.687359,-0.487706,-0.254497,NO_EXPRESSION
1,-0.332848,-0.099455,-0.359186,-0.250329,-0.388346,-0.019337,-0.337193,-0.147278,-0.345062,-0.279849,...,-0.286271,-0.471632,-0.257561,-0.406704,-0.374670,-0.477706,-0.527987,-0.541824,-0.057309,NO_EXPRESSION
2,-0.238044,0.037432,-0.289935,-0.292172,-0.377581,-0.107908,-0.236784,0.011807,-0.243957,-0.119482,...,-0.280596,-0.367891,-0.279142,-0.300270,-0.347517,-0.375597,-0.508254,-0.421590,-0.192883,NO_EXPRESSION
3,-0.293786,0.233733,-0.299647,-0.170271,-0.263565,0.032223,-0.293398,0.204303,-0.295090,0.057314,...,-0.143511,-0.345628,-0.140444,-0.291549,-0.212293,-0.364055,-0.398779,-0.379481,-0.079406,NO_EXPRESSION
4,-0.318250,0.393258,-0.312007,-0.114104,-0.244756,0.063514,-0.320453,0.353198,-0.320885,0.175597,...,-0.126714,-0.346842,-0.127793,-0.280280,-0.187766,-0.384052,-0.413537,-0.350158,-0.104749,NO_EXPRESSION
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,0.315231,-0.430342,0.177588,-0.821165,-0.674301,-1.721223,0.318494,-0.370044,0.305922,-0.482113,...,-0.882902,0.413730,-0.922773,0.370072,-0.917472,0.347736,-0.828713,0.538552,-1.126230,NO_EXPRESSION
4996,0.270355,-0.121272,0.159833,-0.721009,-0.531780,-1.727561,0.270236,-0.036964,0.256739,-0.155476,...,-0.712640,0.399103,-0.784921,0.343566,-0.740851,0.319921,-0.725729,0.587513,-1.105542,NO_EXPRESSION
4997,0.346073,-0.000037,0.182689,-0.711051,-0.527350,-1.559684,0.347879,0.089803,0.332582,-0.026861,...,-0.640794,0.431540,-0.739841,0.384733,-0.665580,0.357021,-0.705977,0.587382,-1.079102,NO_EXPRESSION
4998,0.427783,-0.495542,0.257550,-0.860953,-0.516236,-1.706472,0.431130,-0.416211,0.415139,-0.477445,...,-0.852299,0.477749,-0.920306,0.422376,-0.873770,0.423486,-0.824535,0.630590,-1.164210,NO_EXPRESSION


In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()